In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

from utils import get_completion

model_id = "kreimben/CodeMind-gemma-2b"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

In [ ]:
model = model.to('cuda')

In [ ]:
%%time

text = f'''leetcode wordbreak'''

res = get_completion(
    text,
    system_prompt='You are a kind coding test teacher. Solve the given question and give the insight and approach.',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=4096,
)

In [ ]:
answer = res.split('<|assistant|>')[-1]

In [ ]:
from IPython.display import display, Markdown

# answer = answer.replace('\\\\', '\\')

display(Markdown(answer))